# *Model Training*

### *1.1 Import all the required Libraries*

In [69]:
import os
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [70]:
pwd=os.getcwd()
filepath=pwd+"\Data\StudentsPerformance.csv"
df=pd.read_csv(filepath)

In [71]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [72]:
df["Total_Score"]=df["math score"]+df["reading score"]+df["writing score"]
df["Average"]=df["Total_Score"]/3
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,Total_Score,Average
0,female,group B,bachelor's degree,standard,none,72,72,74,218,72.666667
1,female,group C,some college,standard,completed,69,90,88,247,82.333333
2,female,group B,master's degree,standard,none,90,95,93,278,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,49.333333
4,male,group C,some college,standard,none,76,78,75,229,76.333333


### *Selecting dependent and independent variable*

In [73]:
## Independent variable
X=df.drop(columns=['math score'], axis=1)

In [74]:
X.columns

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'reading score', 'writing score',
       'Total_Score', 'Average'],
      dtype='object')

In [75]:
## Dependent variable
y=df['math score']

In [76]:
### Handling categorical feature.
print("Categories in gender variable: ",end="")
print(df['gender'].unique())

print("Categories in race/enthnicity variable: ",end="")
print(df['race/ethnicity'].unique())

print("Categories in 'parental level of education' variable: ",end="")
print(df['parental level of education'].unique())

print("Categories in lunch variable: ",end="")
print(df['lunch'].unique())

print("Categories in 'test preparation course' varibale: ",end="")
print(df['test preparation course'].unique())

Categories in gender variable: ['female' 'male']
Categories in race/enthnicity variable: ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in 'parental level of education' variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in lunch variable: ['standard' 'free/reduced']
Categories in 'test preparation course' varibale: ['none' 'completed']


### *3.1 Column Transformer* 

In [77]:
### Define numerical and categorical features
numeric_features=[feature for feature in X.columns if df[feature].dtype != 'O']
categorical_features=[feature for feature in X.columns if df[feature].dtype == 'O']

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer= StandardScaler()
category_transformer= OneHotEncoder()

pre_processor= ColumnTransformer(
    [
        ("OneHotEncoder", category_transformer,categorical_features),
        ("StandardScaler", numeric_transformer,numeric_features),
    ]
)

In [78]:
X = pre_processor.fit_transform(X)

In [79]:
X.shape

(1000, 21)

In [80]:
### Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape

((700, 21), (300, 21))

### *Create an evaluation function to give all the metrics after training model*

In [81]:
def evaluate_model(true, predicted):
    mae= mean_absolute_error(true, predicted)
    mse= mean_squared_error(true, predicted)
    rmse= np.sqrt(mean_squared_error(true, predicted))
    r2_square= r2_score(true, predicted)
    return mae, rmse, r2_square

In [82]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [83]:
models={
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train, y_train) # Train model
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.0000
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.0000
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000


Lasso
Model performance for Training set
- Root Mean Squared Error: 4.6538
- Mean Absolute Error: 3.6848
- R2 Score: 0.9018
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4.9626
- Mean Absolute Error: 3.9339
- R2 Score: 0.9010


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.3882
- Mean Absolute Error: 0.3106
- R2 Score: 0.9993
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.4109
- Mean Absolute Error: 0.3274
- R2 Score: 0.9993


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 4.2030
- Mean Absolute Error: 3.3143
- R2 Score: 0.9199
-----------------------

### *Results*

In [84]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
0,Linear Regression,1.000000
2,Ridge,0.999321
5,Random Forest Regressor,0.947994
4,Decision Tree,0.921828
6,AdaBoost Regressor,0.914831
1,Lasso,0.901020
3,K-Neighbors Regressor,0.889137
